In [1]:
# TEMPERATURES New Future 
# OUTS: Temperature per year and per month as one CSV file
# Run only for TG
# could be used as well for Tx and TN, but those are done below

import arcpy
import csv
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]
MonthDay = ["Jan5", "Jan15", "Jan25", "Feb5", "Feb15", "Feb25", "March5", "March15", "March25", "Apr5", "Apr15", "Apr25", \
           "May5", "May15", "May25", "June5", "June15", "June25", "July5", "July15", "July25", "Aug5", "Aug15", "Aug25", \
           "Sept5", "Sept15", "Sept25", "Oct5", "Oct15", "Oct25", "Nov5", "Nov15", "Nov25", "Dec5", "Dec15", "Dec25"]
Folder = r"C:\DannyData\Projects\Agrichange\Climate data\FutureParameters\RCP6_0\RCP6_0HADGEM2 projections Temprature Copernicus\\"
#FolderGrowthSeason = FolderBase + "Mean of daily man temperature\\"
FolderOutPut = "C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\\"

NCInfile1 = Folder + "TG_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20110101-20401231_v1.1.nc"
NCInfile2 = Folder + "TG_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20410101-20701231_v1.1.nc"
NCInfile3 = Folder + "TG_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20710101-20991231_v1.1.nc"
TypeStats = "MEAN"
NameIndicator = "MEAN"
VariableType = "TG"

InputLocations = "LTE-FullSet"
Start_Year = 2075
End_Year = 2100 # note add 1
YearMonthCount = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        OutFileYear = FolderOutPut + "Future_" + NameIndicator + "Temp_PerYear_2"  + ".csv"
        OutFileMonth = FolderOutPut + "Future_" + NameIndicator + "Temp_PerMonth_2"  + ".csv"
        # Extract and Name Days per year
        if year < 2041:
            NCInfile = NCInfile1
        elif year > 2070:
            NCInfile = NCInfile3
        else:
            NCInfile = NCInfile2
        Count = 0
        for date in range(0,len(Days)):
            Count = Count + 1;
            YearMonthCount = YearMonthCount + 1
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmp = "TmpDates"
            LayerName = "Dates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfile, variable=VariableType, x_dimension="lon", y_dimension="lat",
            out_raster_layer=LayerNametmp, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNametmp,in_raster_or_constant2= 273.15)
            out_raster.save(LayerName) 
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName   
                
            TableName = "ZonalTemp"
            if  YearMonthCount == 1:
                TableNametmp = TableName
            else:
                TableNametmp = "TmpTable"           
            
            NewName = "Y_" + str(year) +"_" + MonthDay[date]
            arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= LayerName, out_table=TableNametmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
            arcpy.management.AlterField(in_table=TableNametmp,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
            if YearMonthCount != 1:
                arcpy.management.JoinField(in_data=TableName,in_field="Locations",join_table=TableNametmp,join_field="Locations",
                    fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)            
        
        YearName = "Y_" + str(year)  
        OutMapTemp = FolderOut + "MainDailyTemp_" + "Y_" + str(year) +".tif"      
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("YearFullRast") 
        out_raster.save(OutMapTemp) 
        
        TableYear = "YearTemp"
        if  year == Start_Year:
            Yeartmp = TableYear
        else:
            Yeartmp = "TmpYear"
          
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "YearFullRast", out_table=Yeartmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=Yeartmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableYear,in_field="Locations",join_table=Yeartmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)  
        #end of per LTE loop 
        #Write away per year 
        for i in range(1,2):
            if i == 0:
                table = TableName
                OutFile = OutFileMonth
            else:
                table = TableYear
                OutFile = OutFileYear
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals) 
        arcpy.Delete_management(LayersArrayFull) 
print("Finished                              ")                

Finished                              


In [2]:
# TEMPERATURE RANGE Future
# OUTS: 
# - Min Temperature per year as one CSV file
# - Max temperature per year as one CSV file
# - Mean Daily Temp Range as on CSV file per year (ccalculated here here from TXx and TNn
# - Annual Temperature range as one CSV file (calculated here here from TXx and TNn)
# - Isothermality: = Daily Range/annual range, as one CSV file (calculated here here from TXx and TNn)

# TO Get Layers
import arcpy
import csv
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]
FolderOut = "C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\\"
Folder = "C:\DannyData\Projects\Agrichange\Climate data\FutureClimate\RCP6_0\RCP6_0HADGEM2 projections Temprature Copernicus\\"

NCInfileA1 = Folder + "TXx_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20110101-20401231_v1.1.nc"
NCInfileA2 = Folder + "TXx_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20410101-20701231_v1.1.nc"
NCInfileA3 = Folder + "TXx_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20710101-20991231_v1.1.nc"
NCInfileB1 = Folder + "TNn_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20110101-20401231_v1.1.nc"
NCInfileB2 = Folder + "TNn_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20410101-20701231_v1.1.nc"
NCInfileB3 = Folder + "TNn_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20710101-20991231_v1.1.nc"
TypeStats = "MEAN"
VariableTypeA = "TXx"
VariableTypeB = "TNn"

InputLocations = "LTE-FullSet"
Start_Year = 2090
End_Year = 2100 #note add 1

OutFileDaily = FolderOut + "DailyTempRange" + ".csv"
OutFileMax = FolderOut + "YearMaxTemp" + ".csv"
OutFileMin = FolderOut + "YearMinTemp" + ".csv"
OutFileAnnualRange = FolderOut + "AnnualTempRange" + ".csv"
OutFileIso = FolderOut + "Isothermality" + ".csv"


with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        # Extract and Name Days per year
        if year < 2041:
            NCInfileA = NCInfileA1
            NCInfileB = NCInfileB1
        elif year > 2070:
            NCInfileA = NCInfileA3
            NCInfileB = NCInfileB3
        else:
            NCInfileA = NCInfileA2
            NCInfileB = NCInfileB2
        Count = 0
        for date in range(0,len(Days)):
            Count = Count + 1;
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmpA = "TmpDatesA_" + str(Count)
            LayerNametmpB = "TmpDatesB_" + str(Count)
            LayerNameDaily = "RangeDates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileA, variable=VariableTypeA, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNametmpA, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfileB, variable=VariableTypeB, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNametmpB, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNametmpA,in_raster_or_constant2= LayerNametmpB)
            out_raster.save( LayerNameDaily) 
            # add here per month per LTE and saving

            if date == 0:
                LayersArrayFullDaily =  LayerNameDaily
                LayersArrayFullMaxYear = LayerNametmpA
                LayersArrayFullMinYear = LayerNametmpB
            else:
                LayersArrayFullDaily = LayersArrayFullDaily + ";" +  LayerNameDaily   
                LayersArrayFullMaxYear = LayersArrayFullMaxYear + ";" +  LayerNametmpA
                LayersArrayFullMinYear = LayersArrayFullMinYear + ";" +  LayerNametmpB
        YearName = "Y_" + str(year)                 
        OutMapDaily = FolderOut + "DailyTempRange_" + "Y_" + str(year) +".tif"  
        #Daily Range
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullDaily,
                     statistics_type="MEAN",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("DailyRange") 
        out_raster.save(OutMapDaily) 
        TableYearDaily = "ZonalFullDaily"
        
        if  year == Start_Year:
            Yeartmp = TableYearDaily
        else:
            Yeartmp = "TmpYear"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "DailyRange", out_table=Yeartmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        arcpy.management.AlterField(in_table=Yeartmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableYearDaily,in_field="Locations",join_table=Yeartmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)  
        
        # Maximum
        OutMapMaximum = FolderOut + "MaxTemp_" + "Y_" + str(year) +".tif"    
        TmpMax = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullMaxYear,
                     statistics_type="MAXIMUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster = arcpy.sa.Minus(in_raster_or_constant1=TmpMax,in_raster_or_constant2= 273.15)
        out_raster.save("MaximumTemp") 
        out_raster.save(OutMapMaximum) 
        TableMaxYear = "ZonalMaximum"
        if  year == Start_Year:
            YearMaxtmp = TableMaxYear
        else:
            YearMaxtmp = "TmpMaxYear"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "MaximumTemp", out_table=YearMaxtmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        arcpy.management.AlterField(in_table=YearMaxtmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableMaxYear,in_field="Locations",join_table=YearMaxtmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)  
       
        # Minimum
        OutMapMinimum = FolderOut + "MinTemp_" + "Y_" + str(year)   +".tif"  
        TmpMin = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullMinYear,
                     statistics_type="MINIMUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster = arcpy.sa.Minus(in_raster_or_constant1=TmpMin,in_raster_or_constant2= 273.15)
        out_raster.save("MinimumTemp") 
        out_raster.save(OutMapMinimum) 
        TableMinYear = "ZonalMinimum"
        if  year == Start_Year:
            YearMintmp = TableMinYear
        else:
            YearMintmp = "TmpMinYear"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "MinimumTemp", out_table=YearMintmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        arcpy.management.AlterField(in_table=YearMintmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableMinYear,in_field="Locations",join_table=YearMintmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)      
        
        # Annual Range
        OutMapAnnualRange = FolderOut + "AnnualTempRange_" + "Y_" + str(year)  +".tif"  
        out_raster = arcpy.sa.Minus(in_raster_or_constant1="MaximumTemp",in_raster_or_constant2= "MinimumTemp")
        out_raster.save("AnnualRange") 
        out_raster.save(OutMapAnnualRange) 
        TableRangeYear = "ZonalRange"
        if  year == Start_Year:
            YearRangetmp = TableRangeYear
        else:
            YearRangetmp = "TmpRangeYear"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "AnnualRange", out_table=YearRangetmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        arcpy.management.AlterField(in_table=YearRangetmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableRangeYear,in_field="Locations",join_table=YearRangetmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)      
        
        # Isothermality
        OutMapIso = FolderOut + "Isothermality_" + "Y_" + str(year) +".tif"  
        IsoTemp = arcpy.sa.Divide(in_raster_or_constant1="DailyRange",in_raster_or_constant2= "AnnualRange")
        out_raster = arcpy.sa.Times(in_raster_or_constant1=IsoTemp,in_raster_or_constant2= 100)
        out_raster.save("Isothermality") 
        out_raster.save(OutMapIso) 
        TableIsoYear = "ZonalIso"
        if  year == Start_Year:
            YearIsotmp = TableIsoYear
        else:
            YearIsotmp = "TmpIsoYear"
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "Isothermality", out_table=YearIsotmp ,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        arcpy.management.AlterField(in_table=YearIsotmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableIsoYear,in_field="Locations",join_table=YearIsotmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)     
        
        #end of per LTE loop 
        #Write away per year 
        for i in range(0,5):
            if i == 0:
                table = TableYearDaily
                OutFile = OutFileDaily
            elif i == 1:
                table = TableMaxYear
                OutFile = OutFileMax
            elif i == 2:    
                table = TableMinYear
                OutFile = OutFileMin
            elif i == 3:    
                table = TableRangeYear
                OutFile = OutFileAnnualRange
            elif i == 4:    
                table = TableIsoYear
                OutFile = OutFileIso
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals) 
        arcpy.Delete_management(LayersArrayFullDaily)
        arcpy.Delete_management(LayersArrayFullMaxYear)
        arcpy.Delete_management(LayersArrayFullMinYear )    
print("Finished                              ")    

Finished                              


In [1]:
# Precipitation Future 
# OUTS: Precipitation per year and per month as one CSV file
# Run only for RR

import arcpy
import csv
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]
MonthDay = ["Jan", "Jan", "Jan", "Feb", "Feb", "Feb", "March", "March", "March", "Apr", "Apr", "Apr", \
           "May", "May", "May", "June", "June", "June", "July", "July", "July", "Aug", "Aug", "Aug", \
           "Sept", "Sept", "Sept", "Oct", "Oct", "Oct", "Nov", "Nov", "Nov", "Dec", "Dec", "Dec"]
Folder = r"C:\DannyData\Projects\Agrichange\Climate data\FutureClimate\RCP6_0\RCP6_0HadGEM2 Precipitation Sum Copernicus\\"
FolderOut = "C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\\"

NCInfile1 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20110101-20401231_v1.1.nc"
NCInfile2 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20410101-20701231_v1.1.nc"
NCInfile3 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20710101-20991231_v1.1.nc"
TypeStats = "SUM"
NameIndicator = "SUM"
VariableType = "RR"

InputLocations = "LTE-FullSet"
Start_Year = 2011
End_Year = 2021 # note add 1
YearMonthCount = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        OutFileYear = FolderOut + "Future_" + NameIndicator + "Precipitation_PerYear"  + ".csv"
        OutFileMonth = FolderOut + "Future_" + NameIndicator + "Precipitation_PerMonth"  + ".csv"
        # Extract and Name Days per year
        if year < 2041:
            NCInfile = NCInfile1
        elif year > 2070:
            NCInfile = NCInfile3
        else:
            NCInfile = NCInfile2
        Count = 0
        Combine = 0
        Countthree = 0
        for date in range(0,len(Days)):
            Count = Count + 1;
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmp = "TmpDates"
            LayerName = "Dates_" + str(Count)
            LayerNameThree = "Dates_Month" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfile, variable=VariableType, x_dimension="lon", y_dimension="lat",
            out_raster_layer=LayerNametmp, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNametmp,in_raster_or_constant2= 0)
            out_raster.save(LayerName)
            out_raster.save(LayerNameThree)
            
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName   
            if Countthree == 0:
                LayersArrayFullThree = LayerNameThree
                Countthree = Countthree + 1
            else:
                LayersArrayFullThree = LayersArrayFullThree + ";" + LayerNameThree       
            if (date+1)/3 == round((date+1)/3): # combine per month          
                TableName = "ZonalTemp"
                if  YearMonthCount == 0:
                    TableNametmp = TableName
                else:
                    TableNametmp = "TmpTable"           
                NewName = "Y_" + str(year) +"_" + MonthDay[date]
                out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullThree,
                     statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
                out_raster.save("MonthFullRast") 
                
                arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "MonthFullRast", out_table=TableNametmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
                arcpy.management.AlterField(in_table=TableNametmp,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
                if YearMonthCount != 0:
                    arcpy.management.JoinField(in_data=TableName,in_field="Locations",join_table=TableNametmp,join_field="Locations",
                        fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)
                arcpy.Delete_management(LayersArrayFullThree)
                YearMonthCount = YearMonthCount + 1
                Countthree = 0
        # combine per year     
        YearName = "Y_" + str(year)    
        OutMapRain = FolderOut + "PrecipitationFuture_" + "Y_" + str(year)   +".tif"  
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type=TypeStats,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("YearFullRast") 
        if year > 2010:
            out_raster.save( OutMapRain) 
        
        TableYear = "YearTemp"
        if  year == Start_Year:
            Yeartmp = TableYear
        else:
            Yeartmp = "TmpYear"
        
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "YearFullRast", out_table=Yeartmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=Yeartmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableYear,in_field="Locations",join_table=Yeartmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)  
        #end of per LTE loop 
        #Write away per year 
        for i in range(0,2):
            if i == 0:
                table = TableName
                OutFile = OutFileMonth
            else:
                table = TableYear
                OutFile = OutFileYear
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals)    
        arcpy.Delete_management(LayersArrayFull)
print("Finished                              ")                    

Finished                              


In [2]:
# Climatalogical Rain CV 
# Version full data set Future

# Precipitation New Future
# OUTS: Precipitation per year and per month as one CSV file
# Run only for RR
# Incoperated in BELOw. NOT USE

import arcpy
import csv
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]
MonthDay = ["Jan", "Jan", "Jan", "Feb", "Feb", "Feb", "March", "March", "March", "Apr", "Apr", "Apr", \
           "May", "May", "May", "June", "June", "June", "July", "July", "July", "Aug", "Aug", "Aug", \
           "Sept", "Sept", "Sept", "Oct", "Oct", "Oct", "Nov", "Nov", "Nov", "Dec", "Dec", "Dec"]
Folder = r"C:\DannyData\Projects\Agrichange\Climate data\FutureClimate\RCP6_0\RCP6_0HadGEM2 Precipitation Sum Copernicus\\"
FolderOut = "C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\\"

NCInfile1 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20110101-20401231_v1.1.nc"
NCInfile2 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20410101-20701231_v1.1.nc"
NCInfile3 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20710101-20991231_v1.1.nc"

VariableType = "RR"
OutFileRain = FolderOut + "CVRainFall" + ".csv"
InputLocations = "LTE-FullSet"
Start_Year = 2011
End_Year = 2100 # note add 1

with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Count = 0
        Countthree = 0
        for date in range(0,len(Days)):
            # Extract and Name Days per year
            if year < 2041:
                NCInfile = NCInfile1
            elif year > 2070:
                NCInfile = NCInfile3
            else:
                NCInfile = NCInfile2
            
            Message = "Running Year: " + str(year) + "           "
            print(Message, end="\r")
            Count = Count + 1;
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNameRain = "RainDates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfile, variable=VariableType, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNameRain, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")  
            if Countthree == 0:
                LayersArrayMonth = LayerNameRain
                Countthree = Countthree + 1
            else:
                LayersArrayMonth = LayersArrayMonth + ";" + LayerNameRain       
            
            if (date+1)/3 == round((date+1)/3): # combine per month          
                PerMonthName = "RainDates_" + MonthDay[date]
                out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayMonth,
                     statistics_type="SUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
                out_raster.save(PerMonthName) 
                arcpy.Delete_management(LayersArrayMonth)
                Countthree = 0
                
                if date == 2:
                    LayersArrayFullRain = PerMonthName
                else:
                    LayersArrayFullRain = LayersArrayFullRain + ";" + PerMonthName   
        YearName = "Y_" + str(year)    
        OutMapRain = FolderOut + "Precipitation_" + "Y_" + str(year)   +".tif"              
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullRain,
                     statistics_type="MEAN",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("MEANFull") 
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullRain,
                     statistics_type="STD",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("STDFull")   
        OutMapRain = FolderOut + "CVPrecipitation_" + "Y_" + str(year)   +".tif"      
        out_raster = arcpy.sa.Divide(in_raster_or_constant1="STDFull",in_raster_or_constant2= "MEANFull")
        out_raster.save("RainFallYearMap") 
        if year > 2089:
            out_raster.save(OutMapRain)
        
        TableNameFullRain = "ZonalRain"           
        if  year == Start_Year:
            YeartmpRain = TableNameFullRain
        else:
            YeartmpRain = "TmpYearRain"
        YearName = "Y_" + str(year)        
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "RainFallYearMap", out_table=YeartmpRain,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=YeartmpRain,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableNameFullRain,in_field="Locations",join_table=YeartmpRain,join_field="Locations", 
                         fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)    
            
        #Write away per year 
        table = TableNameFullRain 
        fields = arcpy.ListFields(table)  
        field_names = [field.name for field in fields]  
        with open(OutFileRain,'w') as f:  
            w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
            w.writerow(field_names)  
            for mow in arcpy.SearchCursor(table):  
                field_vals = [mow.getValue(field.name) for field in fields]  
                w.writerow(field_vals)    
print("Finished                               ")

Finished                               


In [ ]:
# Climatalogical PET Sum and CV 
# Version full data set Future

import arcpy
import csv
DaysPET = ["16/01", "15/02", "16/03", "16/04", "16/05", "16/06", "16/07", "16/08", "16/09", "16/10", "16/11", "16/12"]
SecMonth = [2678400, 2419200, 2678400, 2592000, 2678400, 2592000, 2678400,2678400,2592000,2678400,2592000,2678400]
Months = ['Jan','Feb','Mar','Apr','May', 'Jun','Jul','Aug','Sep','Oct','Nov','Dec']

FolderOutPut = r"C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\\"
NCInfilePET =r" C:\DannyData\Projects\Agrichange\Climate data\FutureParameters\RCP6_0\KNMI_Evaporation_CMIP5_rcp6_0.nc" # Per second!
OutFilePETCV = FolderOutPut + "Future_CV_AnnualPET" + ".csv" 
OutFilePETSUM = FolderOutPut + "Future_SUM_AnnualPET" + ".csv" 

VariableTypePET = "evspsbl"
InputLocations = "LTE-FullSet"
Start_Year = 2011
End_YearEntered = 2099
End_Year = arcpy.sa.Plus(in_raster_or_constant1=End_YearEntered,in_raster_or_constant2= 1)

YearMonthCount = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Count = 0
        for date in range(0,len(DaysPET)):
            Message = "Running Year: " + str(year) + " "+ Months[date] + "           "
            print(Message, end="\r")
            Count = Count + 1;
            YearMonthCount = YearMonthCount + 1
            DimensionPET = "time '"+ DaysPET[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNamePET = "PETDates_" + str(Count)
            arcpy.Delete_management("InBetweenPET") 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfilePET, variable=VariableTypePET, x_dimension="lon", y_dimension="lat",
                   out_raster_layer="InBetweenPET", band_dimension="", dimension_values=DimensionPET,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times(in_raster_or_constant1="InBetweenPET",in_raster_or_constant2=SecMonth[date])
            out_raster.save(LayerNamePET)
            
            if date == 0:
                LayersArrayFullPET = LayerNamePET
            else:
                LayersArrayFullPET = LayersArrayFullPET + ";" + LayerNamePET   
        
        YearName = "Y_" + str(year)  
        OutMapPETSUM = FolderOut + "PETSUM_" + "Y_" + str(year)   +".tif"  
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullPET,
                     statistics_type="MEAN",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("MEANFull")
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullPET,
                     statistics_type="SUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("SUMFull")
        if year > 2089:
            out_raster.save(OutMapPETSUM)
        
        OutMapPETCV = FolderOut + "PETCV_" + "Y_" + str(year)   +".tif"  
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullPET,
                     statistics_type="STD",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("STDFull")   
        out_raster = arcpy.sa.Divide(in_raster_or_constant1="STDFull",in_raster_or_constant2= "MEANFull")
        out_raster.save("PETCVMap")
        if year > 2089:
            out_raster.save(OutMapPETCV) 
        
        TableNameCVPET = "ZonalPET"
        TableNameSUMPET = "ZonalPETSum"  
        if  year == Start_Year:
            YeartmpPET = TableNameCVPET
            YeartmpPETSUM = TableNameSUMPET 
        else:
            YeartmpPET = "TmpYearPET"
            YeartmpPETSUM = "TmpYearPETSum"
              
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "SUMFull", out_table=YeartmpPETSUM,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=YeartmpPETSUM,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "PETCVMap", out_table=YeartmpPET,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=YeartmpPET,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableNameCVPET,in_field="Locations",join_table=YeartmpPET,join_field="Locations", 
                         fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)
            arcpy.management.JoinField(in_data=TableNameSUMPET,in_field="Locations",join_table=YeartmpPETSUM,join_field="Locations", 
                         fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)  
            
        #Write away per year 
        table = TableNameCVPET 
        fields = arcpy.ListFields(table)  
        field_names = [field.name for field in fields]  
        with open(OutFilePETCV,'w') as f:  
            w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
            w.writerow(field_names)  
            for mow in arcpy.SearchCursor(table):  
                field_vals = [mow.getValue(field.name) for field in fields]  
                w.writerow(field_vals)
        
        table = TableNameSUMPET 
        fields = arcpy.ListFields(table)  
        field_names = [field.name for field in fields]  
        with open(OutFilePETSUM,'w') as f:  
            w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
            w.writerow(field_names)  
            for mow in arcpy.SearchCursor(table):  
                field_vals = [mow.getValue(field.name) for field in fields]  
                w.writerow(field_vals) 
print("Finished                               ")

In [2]:
# Future Climatalogical water deficit with start at 1-01 every year 
# v4
import arcpy
import csv
DaysPET = ["16/01", "15/02", "16/03", "16/04", "16/05", "16/06", "16/07", "16/08", "16/09", "16/10", "16/11", "16/12"]
SecMonth = [2678400, 2419200, 2678400, 2592000, 2678400, 2592000, 2678400,2678400,2592000,2678400,2592000,2678400]
Months = ['Jan','Feb','Mar','Apr','May', 'Jun','Jul','Aug','Sep','Oct','Nov','Dec']
NCInfilePET =r" C:\DannyData\Projects\Agrichange\Climate data\FutureClimate\RCP6_0\KNMI_Evaporation_CMIP5_rcp6_0.nc" # Per second!
FolderOut = "C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\\"
VariableTypePET = "evspsbl"

DaysRain = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]
MonthDayRain = ["Jan", "Jan", "Jan", "Feb", "Feb", "Feb", "Mar", "Mar", "Mar", "Apr", "Apr", "Apr", \
           "May", "May", "May", "Jun", "Jun", "Jun", "Jul", "Jul", "Jul", "Aug", "Aug", "Aug", \
           "Sep", "Sep", "Sep", "Oct", "Oct", "Oct", "Nov", "Nov", "Nov", "Dec", "Dec", "Dec"]
Folder = r"C:\DannyData\Projects\Agrichange\Climate data\FutureClimate\RCP6_0\RCP6_0HadGEM2 Precipitation Sum Copernicus\\"
NCInfile1 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20110101-20401231_v1.1.nc"
NCInfile2 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20410101-20701231_v1.1.nc"
NCInfile3 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20710101-20991231_v1.1.nc"
VariableTypeRain = "RR"

TypeStatsCWD = "MAXIMUM"
NameIndicator = "SUM"

InputLocations = "LTE-FullSet"
OutFileFull = FolderOut + "FutureCWD_GlobalCWD.csv"

Start_Year = 2090
End_YearEntered = 2099
End_Year = arcpy.sa.Plus(in_raster_or_constant1=End_YearEntered,in_raster_or_constant2= 1)
Previous_Month = 0 
Start_Seq = 0
YearCount = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        YearCount = YearCount + 1
        # Calcuate Rain per Month
        Countthree = 0
        Count = 0
        for date in range(0,len(DaysRain)):
            # Extract and Name Days per year
            if year < 2041:
                NCInfile = NCInfile1
            elif year > 2070:
                NCInfile = NCInfile3
            else:
                NCInfile = NCInfile2
            
            Message = "Running Rain for Year: " + str(year) + "           "
            print(Message, end="\r")
            Count = Count + 1;
            Dimension = "time '"+ DaysRain[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNameRain = "RainDates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfile, variable=VariableTypeRain, x_dimension="lon", y_dimension="lat",
                   out_raster_layer=LayerNameRain, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")  
            if Countthree == 0:
                LayersArrayMonth = LayerNameRain
                Countthree = Countthree + 1
            else:
                LayersArrayMonth = LayersArrayMonth + ";" + LayerNameRain       
            
            if (date+1)/3 == round((date+1)/3): # combine per month          
                PerMonthName = "RainDates_" + MonthDayRain[date]
                out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayMonth,
                     statistics_type="SUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
                out_raster.save(PerMonthName) 
                arcpy.Delete_management(LayersArrayMonth)
                Countthree = 0
        # Water deficit per calender year, so every year a new one with zeros is generated    
        arcpy.Delete_management("Previous_MonthDeficit") 
        with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', scratchWorkspace=r"C:\DannyData\Projects\Agrichange\ArcPro project\AgriChange\AgriChange.gdb"):
            out_raster = arcpy.sa.CreateConstantRaster(constant_value=0,data_type="INTEGER",cell_size=0.25,extent="-180 -90 180 90")
            out_raster.save("Previous_MonthDeficit")
        Count = 0
        for date in range(0,len(DaysPET)):
            Message = "Running CWD for Year: " + str(year) + "           "
            print(Message, end="\r")
            Count = Count + 1;
            DimensionPET = "time '"+ DaysPET[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNamePET = "PETDates_" + str(Count)
            LayerName = "Dates_" + str(Count)
            arcpy.Delete_management("InBetweenPET") 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfilePET, variable=VariableTypePET, x_dimension="lon", y_dimension="lat",
                   out_raster_layer="InBetweenPET", band_dimension="", dimension_values=DimensionPET,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times(in_raster_or_constant1="InBetweenPET",in_raster_or_constant2=SecMonth[date])
            out_raster.save(LayerNamePET)
            
            RainPerMonthName = "RainDates_" + Months[date]
            with arcpy.EnvManager(cellSize=RainPerMonthName):
                out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNamePET,in_raster_or_constant2= RainPerMonthName)
                out_raster.save("InBetweenCMW")
            out_raster = arcpy.sa.Plus(in_raster_or_constant1="InBetweenCMW",in_raster_or_constant2= "Previous_MonthDeficit")
            out_raster.save("InBetweenCMW2")
            arcpy.Delete_management("Previous_MonthDeficit") 
            out_raster = arcpy.sa.Con(in_conditional_raster="InBetweenCMW2",in_true_raster_or_constant=0,
                         in_false_raster_or_constant="InBetweenCMW2",where_clause="VALUE < 0")
            out_raster.save(LayerName)
            arcpy.management.CopyRaster(in_raster=LayerName,out_rasterdataset="Previous_MonthDeficit",config_keyword="",background_value=None,
                                        nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                                        RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                                        build_multidimensional_transpose="NO_TRANSPOSE")            
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName   
        OutMapCWD = FolderOut + "CWDFuture_" + "Y_" + str(year) +".tif" 
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type=TypeStatsCWD,ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("CWDSumFull")
        out_raster.save(OutMapCWD)
        TableNameFull = "ZonalCWD"
        if  YearCount == 1:
            TableNametmp = TableNameFull
        else:
            TableNametmp = "TmpSeasonYear"
        with arcpy.EnvManager(overwriteOutput=True,cellSize=0.005):
            arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "CWDSumFull", out_table=TableNametmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
        NewName = "Y_" + str(year)
        arcpy.management.AlterField(in_table=TableNametmp,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR") 
        if YearCount != 1:
            arcpy.management.JoinField(in_data=TableNameFull,in_field="Locations",join_table=TableNametmp,join_field="Locations",
                    fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)   

        #end of per LTE loop 
        table = TableNameFull 
        OutFile = OutFileFull
        fields = arcpy.ListFields(table)  
        field_names = [field.name for field in fields]  
        with open(OutFile,'w') as f:  
            w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
            w.writerow(field_names)  
            for mow in arcpy.SearchCursor(table):  
                field_vals = [mow.getValue(field.name) for field in fields]  
                w.writerow(field_vals)      

In [1]:
# Climatalogical evaporation per MONTH and PER YEAR FUTURE; COULD BE USED FOR CWD
# CWD is not calcuated here but in Matlab and after Evaporation reallignment with other data to PET

# Outs: 
# - Monthly evaporartion as one CSV file, to be used for CWD calculations after realligment
# - Annual evaporation as one CSV file, needs reallignment


import arcpy
import csv
DaysPET = ["16/01", "15/02", "16/03", "16/04", "16/05", "16/06", "16/07", "16/08", "16/09", "16/10", "16/11", "16/12"]
SecMonth = [2678400, 2419200, 2678400, 2592000, 2678400, 2592000, 2678400,2678400,2592000,2678400,2592000,2678400]
Months = ['Jan','Feb','Mar','Apr','May', 'Jun','Jul','Aug','Sep','Oct','Nov','Dec']

FolderOutPut = r"C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\\"
NCInfilePET =r" C:\DannyData\Projects\Agrichange\Climate data\FutureParameters\RCP6_0\KNMI_Evaporation_CMIP5_rcp6_0.nc"

VariableTypePET = "evspsbl"

InputLocations = "LTE-FullSet"
Start_Year = 2011
End_YearEntered = 2099
End_Year = arcpy.sa.Plus(in_raster_or_constant1=End_YearEntered,in_raster_or_constant2= 1)

YearMonthCount = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        OutFilePET = FolderOutPut + "FutureMonthlyPET" + ".csv"
        OutFilePETYear = FolderOutPut + "FutureAnnualPET" + ".csv"
        PETSumLayer = "SumPET"    
        Count = 0
        for date in range(0,len(DaysPET)):
            Message = "Running Year: " + str(year) + " "+ Months[date] + "           "
            print(Message, end="\r")
            Count = Count + 1;
            YearMonthCount = YearMonthCount + 1
            DimensionPET = "time '"+ DaysPET[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmpPET = "TmpDatesPET" 
            arcpy.Delete_management("InBetweenPET") 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfilePET, variable=VariableTypePET, x_dimension="lon", y_dimension="lat",
                   out_raster_layer="InBetweenPET", band_dimension="", dimension_values=DimensionPET,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times(in_raster_or_constant1="InBetweenPET",in_raster_or_constant2=SecMonth[date])
            out_raster.save(LayerNametmpPET)
            
            if Count == 1:
                arcpy.Delete_management(PETSumLayer)
                arcpy.management.CopyRaster(in_raster=LayerNametmpPET,out_rasterdataset=PETSumLayer,config_keyword="",background_value=None,
                      nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                      RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                      build_multidimensional_transpose="NO_TRANSPOSE")   
            else:
                out_raster = arcpy.sa.Plus(in_raster_or_constant1=PETSumLayer,in_raster_or_constant2= LayerNametmpPET)
                out_raster.save("InBetweenPETSum")
                arcpy.Delete_management(PETSumLayer)
                arcpy.management.CopyRaster(in_raster="InBetweenPETSum",out_rasterdataset=PETSumLayer,config_keyword="",background_value=None,
                      nodata_value="",onebit_to_eightbit="NONE",colormap_to_RGB="NONE", pixel_type="",scale_pixel_value="NONE",
                      RGB_to_Colormap="NONE",format="",transform="NONE",process_as_multidimensional="CURRENT_SLICE",
                      build_multidimensional_transpose="NO_TRANSPOSE")   
                arcpy.Delete_management("InBetweenPETSum")
            
            TableNameFullPET = "ZonalPET"
            if  YearMonthCount == 1:
                TableNametmpPET = TableNameFullPET
            else:
                TableNametmpPET = "TmpPET"
            
            NewName = "Y_" + str(year) +"_" + Months[date]  
            arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= LayerNametmpPET, out_table=TableNametmpPET,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)
            arcpy.management.AlterField(in_table=TableNametmpPET,field="MEAN",new_field_name=NewName,new_field_alias=NewName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
            if  YearMonthCount != 1:  
                arcpy.management.JoinField(in_data=TableNameFullPET,in_field="Locations",join_table=TableNametmpPET,join_field="Locations",
                    fields=NewName,fm_option="NOT_USE_FM",field_mapping=None)
                
        TableYearPET = "YearPET"
        if  year == Start_Year:
            YeartmpPET = TableYearPET
        else:
            YeartmpPET = "TmpYearPET"
        YearName = "Y_" + str(year)        
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= PETSumLayer, out_table=YeartmpPET,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=YeartmpPET,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year: 
            arcpy.management.JoinField(in_data=TableYearPET,in_field="Locations",join_table=YeartmpPET,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)    
        
         #Write away per year 
        for i in range(0,2):
            if i == 0:
                table = TableNameFullPET
                OutFile = OutFilePET  
            else:
                table = TableYearPET
                OutFile = OutFilePETYear
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]  
                    w.writerow(field_vals)    
print("Finished                              ")                    

Finished                              


In [1]:
# Radiation Future 
# Needs realignment with POWER prodictions
# Do seperealty for Mean and Max (among month) radiation 
# Outs: Radiation (mean or max) per year in one CSV


import arcpy
import csv
Days = ["16/01", "15/02", "16/03", "16/04", "16/05", "16/06", "16/07", "16/08", "16/09", "16/10", "16/11", "16/12"] #Europe

NCInfile = r"C:\DannyData\Projects\Agrichange\Climate data\FutureParameters\RCP6_0\KNMI_RSDS_CMIP5_rcp6_0..nc" #Africa
TypeStats = "MEAN"
VariableType = "rsds" 
InputLocations = "LTE-FullSet"
Start_Year = 2031
End_Year = 2100 # note add 1

with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        OutFileYear = r"C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\FutureMeanRadation_PerYear2.csv" #Mean
        OutFileYearMax = r"C:\DannyData\Projects\Agrichange\Climate data\TemporaryOutputs\FutureMaxRadation_PerYear2.csv" #Maximum
        # Extract and Name Days per year
        Count = 0
        for date in range(0,len(Days)):
            Count = Count + 1;
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerName = "Dates_" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfile, variable=VariableType, x_dimension="lon", y_dimension="lat",
            out_raster_layer="LayerNameTmp", band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times("LayerNameTmp", 1)
            out_raster.save(LayerName)
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName   
        
        # Mean Radiation        
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type="MEAN",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("YearFullRast") 
        TableYear = "YearTemp"
        if  year == Start_Year:
            Yeartmp = TableYear
        else:
            Yeartmp = "TmpYear"
        YearName = "Y_" + str(year)    
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "YearFullRast", out_table=Yeartmp,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=Yeartmp,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableYear,in_field="Locations",join_table=Yeartmp,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)  
       
        # Max Radiation    
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFull,
                     statistics_type="MAXIMUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save("YearFullRastMax")  
        TableYearMax = "YearTempMax"
        if  year == Start_Year:
            YeartmpMax = TableYearMax
        else:
            YeartmpMax = "TmpYearMax"
        YearName = "Y_" + str(year)    
        arcpy.sa.ZonalStatisticsAsTable(in_zone_data=InputLocations, zone_field="Locations",in_value_raster= "YearFullRastMax", out_table=YeartmpMax,
                    ignore_nodata="DATA",statistics_type="MEAN",process_as_multidimensional="CURRENT_SLICE",percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",circular_calculation="ARITHMETIC",circular_wrap_value=360)        
        arcpy.management.AlterField(in_table=YeartmpMax,field="MEAN",new_field_name=YearName,new_field_alias=YearName,
                         field_type="DOUBLE",field_length=8,field_is_nullable="NULLABLE",clear_field_alias="DO_NOT_CLEAR")
        if year != Start_Year:
            arcpy.management.JoinField(in_data=TableYearMax,in_field="Locations",join_table=YeartmpMax,join_field="Locations",
                 fields=YearName,fm_option="NOT_USE_FM",field_mapping=None)      
        
        arcpy.Delete_management(LayersArrayFull)  
        #end of per LTE loop 
        #Write away per year 
        for i in range(0,2):
            if i == 0:
                table = TableYear
                OutFile = OutFileYear
            else:
                table = TableYearMax
                OutFile = OutFileYearMax
            fields = arcpy.ListFields(table)  
            field_names = [field.name for field in fields]  
            with open(OutFile,'w') as f:  
                w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                w.writerow(field_names)  
                for mow in arcpy.SearchCursor(table):  
                    field_vals = [mow.getValue(field.name) for field in fields]    
                    w.writerow(field_vals)    
print("Finished                                                                   ")

Finished                                                                   


In [2]:
# For multi year averages for Rainfall changes for figures based on one (future) data set source
# Precipitation New Future
# OUTS: Precipitation per year and per month as one CSV file
# Run only for RR


import arcpy
import csv
Days = ["05/01", "15/01", "25/01", "05/02", "15/02", "25/02", "05/03", "15/03", "25/03", "05/04", "15/04", "25/04", \
         "05/05", "15/05", "25/05", "05/06", "15/06", "25/06", "05/07", "15/07", "25/07", "05/08", "15/08", "25/08", \
         "05/09", "15/09", "25/09", "05/10", "15/10", "25/10", "05/11", "15/11", "25/11", "05/12", "15/12", "25/12"]
MonthDay = ["Jan", "Jan", "Jan", "Feb", "Feb", "Feb", "March", "March", "March", "Apr", "Apr", "Apr", \
           "May", "May", "May", "June", "June", "June", "July", "July", "July", "Aug", "Aug", "Aug", \
           "Sept", "Sept", "Sept", "Oct", "Oct", "Oct", "Nov", "Nov", "Nov", "Dec", "Dec", "Dec"]
Folder = r"C:\DannyData\Projects\Agrichange\Climate data\FutureClimate\RCP6_0\RCP6_0HadGEM2 Precipitation Sum Copernicus\\"

NCInfile1 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20110101-20401231_v1.1.nc"
NCInfile2 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20410101-20701231_v1.1.nc"
NCInfile3 = Folder + "RR_C3S-glob-agric_hadgem2-es_rcp6p0_dek_20710101-20991231_v1.1.nc"
TypeStats = "SUM"
NameIndicator = "SUM"
VariableType = "RR"

InputLocations = "LTE-FullSet"
Start_Year = 2011
End_Year = 2020 # note add 1
CountYear = 0;
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        CountYear = CountYear + 1;
        Message = "Running Year: " + str(year) + "           "
        print(Message, end="\r")
        # Extract and Name Days per year
        if year < 2041:
            NCInfile = NCInfile1
        elif year > 2070:
            NCInfile = NCInfile3
        else:
            NCInfile = NCInfile2
        Count = 0
        Countthree = 0
        WithinYearCount = 0;
        for date in range(0,len(Days)):
            Count = Count + 1;
            Dimension = "time '"+ Days[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmp = "TmpDates"
            LayerName = "Dates_" + str(Count)
            LayerNameThree = "Dates_Month" + str(Count)
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfile, variable=VariableType, x_dimension="lon", y_dimension="lat",
            out_raster_layer=LayerNametmp, band_dimension="", dimension_values=Dimension,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Minus(in_raster_or_constant1=LayerNametmp,in_raster_or_constant2= 0)
            out_raster.save(LayerName)
            
            if date == 0:
                LayersArrayFull = LayerName
            else:
                LayersArrayFull = LayersArrayFull + ";" + LayerName 
        # combine per year
        YearName = "RainFallY_" + str(year)       
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants= LayersArrayFull,
                     statistics_type="SUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")
        out_raster.save(YearName) 
        
        if CountYear == 1:
             LayersArrayYears =YearName
        else:
             LayersArrayYears = LayersArrayYears + ";" + YearName 
    out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayYears,
                     statistics_type="MEAN",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")   
    out_raster.save(r"C:\DannyData\Projects\Agrichange\Climate data\RainFall11_20v2.tif")     
print("Finished                              ")                

Finished                              


In [5]:
# For multi year averages for PET changes for figures based on one (future) data set source
# Climatalogical evaporation per MONTH and PER YEAR FUTURE; COULD BE USED FOR CWD
# CWD is not calcuated here but in Matlab and after Evaporation reallignment with other data to PET

# Outs: 
# - Monthly evaporartion as one CSV file, to be used for CWD calculations after realligment
# - Annual evaporation as one CSV file, needs reallignment

# V3.1

import arcpy
import csv
DaysPET = ["16/01", "15/02", "16/03", "16/04", "16/05", "16/06", "16/07", "16/08", "16/09", "16/10", "16/11", "16/12"]
SecMonth = [2678400, 2419200, 2678400, 2592000, 2678400, 2592000, 2678400,2678400,2592000,2678400,2592000,2678400]
Months = ['Jan','Feb','Mar','Apr','May', 'Jun','Jul','Aug','Sep','Oct','Nov','Dec']

NCInfilePET =r"C:\DannyData\Projects\Agrichange\Climate data\FutureClimate\RCP6_0\KNMI_Evaporation_CMIP5_rcp6_0.nc" # Per second!
VariableTypePET = "evspsbl"

InputLocations = "LTE-FullSet"
Start_Year = 2090
End_YearEntered = 2099
End_Year = arcpy.sa.Plus(in_raster_or_constant1=End_YearEntered,in_raster_or_constant2= 1)

CountYear = 0
with arcpy.EnvManager(overwriteOutput=True):
    for year in range(Start_Year,End_Year):
        CountYear = CountYear + 1
        Count = 0
        for date in range(0,len(DaysPET)):
            Message = "Running Year: " + str(year) + " "+ Months[date] + "           "
            print(Message, end="\r")
            Count = Count + 1;
            DimensionPET = "time '"+ DaysPET[date] +"/" +str(year) +"  12:00:00 AM'"  
            LayerNametmpPET = "PETDates_" + str(Count) 
            arcpy.Delete_management("InBetweenPET") 
            arcpy.md.MakeNetCDFRasterLayer(in_netCDF_file=NCInfilePET, variable=VariableTypePET, x_dimension="lon", y_dimension="lat",
                   out_raster_layer="InBetweenPET", band_dimension="", dimension_values=DimensionPET,
                   value_selection_method="BY_VALUE", cell_registration="CENTER")
            out_raster = arcpy.sa.Times(in_raster_or_constant1="InBetweenPET",in_raster_or_constant2=SecMonth[date])
            out_raster.save(LayerNametmpPET)
            
            if date == 0:
                LayersArrayFullPET = LayerNametmpPET
            else:
                 LayersArrayFullPET =  LayersArrayFullPET + ";" + LayerNametmpPET 
        YearName = "PETY_" + str(year)              
        with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', snapRaster=None, extent='-217.222823048928 -113.943744937245 242.018604784816 135.027885149131 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', mask=None, scratchWorkspace=r"C:\DannyData\Projects\Agrichange\ArcPro project\AgriChange\AgriChange.gdb"):
            out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayFullPET,
                     statistics_type="SUM",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")  
            out_raster.save(YearName)     
            if CountYear == 1:
                LayersArrayYears =YearName
            else:
                LayersArrayYears = LayersArrayYears + ";" + YearName 
    with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', snapRaster=None, extent='-217.222823048928 -113.943744937245 242.018604784816 135.027885149131 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', mask=None, scratchWorkspace=r"C:\DannyData\Projects\Agrichange\ArcPro project\AgriChange\AgriChange.gdb"):
        out_raster = arcpy.sa.CellStatistics(in_rasters_or_constants=LayersArrayYears,
                     statistics_type="MEAN",ignore_nodata="DATA",process_as_multiband="SINGLE_BAND",
                     percentile_value=90,percentile_interpolation_type="AUTO_DETECT")   
        out_raster.save(r"C:\DannyData\Projects\Agrichange\Climate data\PETKNMI90_99.tif")     
print("Finished                              ")                    

Finished                              
